# **LTAT.02.001 Neural Networks Mid term exam**
<span style="color:red;">Date: April 2, 2024 </span>

<div style="background-color:rgba(0, 0, 0, 0.0470588); text-align:center; vertical-align: middle; padding:40px 0;">
<a href="/ID">Student ID: FILL THIS IN!</a>
</div>

**To start working, create your copy 'File-> Save a copy in Drive' and work in your saved copy**

**You need to submit the solved notebook through https://courses.cs.ut.ee/2024/nn/spring/Main/Tests**

The exam is in two parts; programming and answering theoritical questions. 
The programming section includes basics of Neural Networks; feed forward network, backpropagation algorithm, and the theoretical concepts you have learned during the course.

**You are eligible for partial points provided you support your code with enough explanation in the comment blocks**

# Part I (20 points)

In this problem, we will use a simple feed forward neural network to predict the probability of having brown eyes given two inputs: age, height. In other words, we have two inputs and one output predicting the **probability** of having brown eyes.
In the following tasks, you need to do once forward-pass and backward-pass using a simple neural network which has an input layer with three input neurons, a hidden layer with three hidden neurons and an output layer with one output neuron.


### Neural Network Initial Values


*   Input Neurons: $x_1, x_2$
*   Hidden Neurons: $h_1, h_2$
*   Output Neurons: $o_1$
*   Activation Function: custom activation function (see the formula below) in the hidden layer and no activation in the output layer.
*   Initial weights in the first layer:
    *  $W_{11}^{(1)} = 0.25, \ \  \quad W_{12}^{(1)} = 0.15$
    *  $W_{21}^{(1)} = 0.10,\quad W_{22}^{(1)} = 0.1$
    *  $W_{ij}^{(1)}$ denotes the parameter between the **ith** input neuron $x_i$ and the **jth** hidden neuron $h_j$.
*   Initial weights in the second layer:
    *  $W_{11}^{(2)} = 0.25$
    *  $W_{21}^{(2)} = 0.35$
    *  $W_{ij}^{(2)}$ denotes the parameter between the **ith** hidden neuron $h_i$ and the **jth** output neuron $o_j$.
*   Initial learning rate is **0.1**.
*   Target value: $t_1 = 0.6$
*   The hidden and output neurons include a bias ($b_1^{(1)}, b_2^{(1)}$ for hidden and $b_1^{(2)}$ for output layer) of 0.
    *  Do not forget to update their weights as well.
*   We will use the **MSE** loss as the loss function.

Hence we have only one output, **MSE loss** is

$$L_{MSE}=(y - \hat{y})^2$$

where $\hat{y}$ is the output prediction and $y$ is the target value. We are using natural logarithm for $\log$ operations.

Here are some necessary derivatives for the backpropagation algorithm.

**Derivative of  MSE loss**

$$\frac{\partial L_{MSE}}{\partial \hat{y}} = -2(y-\hat{y})$$



**Custom activation function in the hidden layer**:
$$
y = \log(1+e^x)
$$ &nbsp;

**Derivative for activation function in the hidden layer**:
$$
\frac{\partial y}{\partial x} = \sigma(x)
$$

where $\sigma(x)$ is the sigmoid function.


### Task 1.1 - Activation Function and Loss (5 points)

Implement activation function, the corresponding derivative and loss function. 

In [1]:
import numpy as np

def activation_function(t):
  cache = t
  return np.log(1 + np.exp(-t)), cache

def activation_function_derivative(t):
  return 1 / (1 + np.exp(-t))

def loss(P, Y):
  return np.square(Y - P)

def loss_derivative(P, Y):
  return -2 * (Y - P)

In [2]:
def affine_forward(x, w, b):
    """
    Computes the forward pass for an affine (fully-connected) layer.

    The input x has shape (N, d_1, ..., d_k) and contains a minibatch of N
    examples, where each example x[i] has shape (d_1, ..., d_k). For example,
    batch of 500 RGB CIFAR-10 images would have shape (500, 32, 32, 3). We 
    will reshape each input into a vector of dimension D = d_1 * ... * d_k,
    and then transform it to an output vector of dimension M.

    Inputs:
    - x: A numpy array containing input data, of shape (N, d_1, ..., d_k)
    - w: A numpy array of weights, of shape (D, M)
    - b: A numpy array of biases, of shape (M,)

    Returns a tuple of:
    - out: output, of shape (N, M)
    - cache: (x, w, b)
    """
    out = None
    ###########################################################################
    # TODO: Implement the affine forward pass. Store the result in out. You   #
    # will need to reshape the input into rows.                               #
    ###########################################################################
    out = x.reshape(x.shape[0], np.prod(x.shape[1:])) @ w + b
    ###########################################################################
    #                             END OF YOUR CODE                            #
    ###########################################################################
    cache = (x, w, b)
    return out, cache


def affine_backward(dout, cache):
    """
    Computes the backward pass for an affine layer.

    Inputs:
    - dout: Upstream derivative, of shape (N, M)
    - cache: Tuple of:
      - x: Input data, of shape (N, d_1, ... d_k)
      - w: Weights, of shape (D, M)

    Returns a tuple of:
    - dx: Gradient with respect to x, of shape (N, d1, ..., d_k)
    - dw: Gradient with respect to w, of shape (D, M)
    - db: Gradient with respect to b, of shape (M,)
    """
    x, w, b = cache
    dx, dw, db = None, None, None
    ###########################################################################
    # TODO: Implement the affine backward pass. Do not forget to reshape your #
    # dx to match the dimensions of x.                                        #
    ###########################################################################
    dx = np.dot(dout, w.T).reshape(x.shape)
    dw = np.dot(x.reshape(x.shape[0], np.prod(x.shape[1:])).T, dout)
    db = np.sum(dout, axis=0)
    ###########################################################################
    #                             END OF YOUR CODE                            #
    ###########################################################################
    assert dx.shape == x.shape, "dx.shape != x.shape: " + str(dx.shape) + " != " + str(x.shape)
    assert dw.shape == w.shape, "dw.shape != w.shape: " + str(dw.shape) + " != " + str(w.shape)
    assert db.shape == b.shape, "db.shape != b.shape: " + str(db.shape) + " != " + str(b.shape)

    return dx, dw, db


In [3]:
def affine_custom_activation_forward(x, w, b, verbose=False):

    a, fc_cache = affine_forward(x, w, b)
    if verbose:
        print(f"hidden input: {a}")
    out, arctan_cache = activation_function(a)
    if verbose:
        print(f"hidden output: {out}")
    cache = (fc_cache, arctan_cache)
    return out, cache

def affine_custom_activation_backward(dout, cache):
    fc_cache, activation_fun_cache = cache
    da = dout * activation_function_derivative(activation_fun_cache)
    dx, dw, db = affine_backward(da, fc_cache)
    return dx, dw, db

In [4]:
class TwoLayerNet(object):

    def __init__(self, W1, b1,  W2, b2, verbose=False):

        self.params = {}
        
        self.params['W1'] = W1
        self.params['b1'] = b1
        self.params['W2'] = W2
        self.params['b2'] = b2
        self.num_layers = 2

        self.verbose = verbose

    def loss(self, X, y=None):

        scores = None
        hidden, cache1 = affine_custom_activation_forward(X, self.params['W1'], self.params['b1'], self.verbose)
        scores, cache2 = affine_forward(hidden, self.params['W2'], self.params['b2'])

        # If y is None then we are in test mode so just return scores
        if y is None:
            return scores

        loss_val, grads = 0, {}
        
        probs = scores

        # Add regularization to the loss
        loss_val = loss(probs, y)

        dscores = loss_derivative(probs, y) / y.shape[0]

        # Backprop into the second layer
        dhidden, dW2, db2 = affine_backward(dscores, cache2)
        grads[f'W{self.num_layers}'] = dW2
        grads[f'b{self.num_layers}'] = db2

        # Backprop into the first layer
        dx, dW1, db1 = affine_custom_activation_backward(dhidden, cache1)
        grads['W1'] = dW1
        grads['b1'] = db1

        if self.verbose:
            print(f'dx: {dx}')

        return loss_val, grads

### Task 1.2 - Forward Pass (5 points)

Perform a forward pass using inputs $x_1=0.3,x_2=0.6$ and given initial weight values, then update the following tables.

* Input: The net input going in the corresponding neuron.
* Output: The net output of the corresponding neuron. **Hint:** Activation function.
* Loss (L): The loss calculated after a forward-pass.

For all responses in table keep at least 3 digit precision.

| Unit  | Input of unit | Output of unit |
|-------|---------------|----------------|
| $h_1$ |  0.135         | 0.628          |
| $h_2$ |    0.105      | 0.642     |
| $o_1$ | 0.382 | 0.382          |


| Loss (L) |
|----------|
| 0.048    |


You can solve the forward and backward pass tasks whichever way you like, you can:
 * Write everything in code with matrix multiplications etc, we have provided a code block for you, but feel free to write your own code.
 * Use code to do calculations one-by-one for each variable. This is basically solving it by hand, but using python as calculator. If you choose this option please make sure that you use reasonable and understandable variable names and calculation order and don't forget to comment.
 * Use actual pen and paper to write down the calculations. In this case we need you to include images of the calculations. If your handwriting is messy or not understandable, you will lose points. If possible, prefer one of the other options.

In [5]:
###########
# You can use this code block for your work or feel free to write your own code.

# You may start by defining matrices
W1 = np.array([[0.25, 0.15],[0.1, 0.10]])
W2 = np.array([[0.25], [0.35]])
b1 = np.array([0,0])
b2 = np.array([0])
learning_rate = 0.1

# dataset
X = np.array([[0.3, 0.6]])
y = np.array([[0.6]])

#####################
## FORWARD AND LOSS##
#####################

model = TwoLayerNet(W1, b1, W2, b2, verbose=True)

score = model.loss(X)
print(f"scores: {score}")

l, grads = model.loss(X, y)
print(f"loss: {l}")

#####################

hidden input: [[0.135 0.105]]
hidden output: [[0.62792358 0.64202467]]
scores: [[0.38168953]]
hidden input: [[0.135 0.105]]
hidden output: [[0.62792358 0.64202467]]
dx: [[-0.02662647 -0.01386725]]
loss: [[0.04765946]]


<font color='red'>Report your results in the table above

### Task 1.3 - Backward Pass (10 points)

Perform a backward pass using the loss calculated above and the parameters provided in the beginning, then, update the following tables.

#### Second Layer (3 p)

Your task here is to calculate delta terms or errors for the given parameters. Note that weights belong to the second layer.

| Parameter  | Initial value | Derivative of loss wrt the parameter | Updated value |
|------------|---------------|--------------------------------------|---------------|
| $W_{11}^{(2)}$ | 0.25 | -0.274                               | 0.277         |
| $W_{21}^{(2)}$ | 0.35 | -0.280                               | 0.378         |
| $b_1^{(2)}$    | 0 | -0.437                               | 0.044         |

#### First Layer (4 p)

Your task here is to calculate delta terms or errors for the given parameters. Note that weights belong to the first layer.

| Parameter  | Initial value | Derivative of loss wrt the parameter | Updated value |
|------------|---------------|--------------------------------------|---------------|
| $W_{12}^{(1)}$ | 0.15 | -0.024                               | 0.152         |
| $W_{22}^{(1)}$ | 0.1 | -0.048                               | 0.105         |
| $b_2^{(1)}$    | 0 |      -0.080                                |      0.008         |


#### Input Layer (3 p)

Your task here is to calculate delta terms or errors with respect to input neurons

**We do not perform update on input**

| Parameter  | Initial value | Derivative of loss wrt the parameter | Updated value |
|------------|---------------|--------------------------------------|--------------|
| $x_{1}$ | 0.3 | -0.027                               | n/a          |
| $x_{2}$ | 0.6 | -0.014                               | n/a          |

In [6]:
###################
## BACKWARD Pass ##
###################

l, grads = model.loss(X, y)
print(f"loss: {l}")


hidden input: [[0.135 0.105]]
hidden output: [[0.62792358 0.64202467]]
dx: [[-0.02662647 -0.01386725]]
loss: [[0.04765946]]


In [7]:
###########
## Update values
###########

print(f"W1_12: {model.params['W1'][0][1]}")
print(f"W1_12: {model.params['W1'][1][1]}")

print(f"dW1_12: {grads['W1'][0][1]}")
print(f"dW1_22: {grads['W1'][1][1]}")

W1_upd = model.params['W1']
W1_upd -= learning_rate * grads['W1']

print(f"W1_12_upd: {W1_upd[0][1]}")
print(f"W1_22_upd: {W1_upd[1][1]}")

print(f"db1_2: {grads['b1'][1]}")

b1_upd = model.params['b1']

b1_upd = b1_upd - learning_rate * grads['b1']

print(f"b1_2_upd: {b1_upd[1]}")

# W2

print(f"W2_12: {model.params['W2'][0][0]}")
print(f"W2_12: {model.params['W2'][1][0]}")

print(f"dW2_12: {grads['W2'][0][0]}")
print(f"dW2_22: {grads['W2'][1][0]}")

W2_upd = model.params['W2']
W2_upd -= learning_rate * grads['W2']

print(f"W2_12_upd: {W2_upd[0][0]}")
print(f"W2_22_upd: {W2_upd[1][0]}")

print(f"db2_2: {grads['b2'][0]}")

b2_upd = model.params['b2']

b2_upd = b2_upd - learning_rate * grads['b2']

print(f"b2_2_upd: {b2_upd[0]}")



W1_12: 0.15
W1_12: 0.1
dW1_12: -0.024124931380783286
dW1_22: -0.04824986276156657
W1_12_upd: 0.15241249313807834
W1_22_upd: 0.10482498627615666
db1_2: -0.08041643793594429
b1_2_upd: 0.008041643793594429
W2_12: 0.25
W2_12: 0.35
dW2_12: -0.2741645827984359
dW2_22: -0.2803214162587189
W2_12_upd: 0.27741645827984357
W2_22_upd: 0.37803214162587184
db2_2: -0.43662094008203256
b2_2_upd: 0.043662094008203256


<font color='red'>**!!! Remember to fill answer tables and explain how you get there by either attaching picture of your solution (or giving us your paper solution) OR commenting your code and explaining your prints!!!**</font> If your answer is slightly wrong, but you reported correct logic, you still can get points!

Also make sure to check indexes of values that we ask to report in table, as we are not asking to report all values from the first layer!

# Part II (10 points)

---


Please answer the following questions in a few sentences.

1. Does it make sense to update the input layer in the above application? why? (2 pts)
- **Ans:** 
- Input layer is fixed, unless we're allowed to change our dataset dimentions (e.g. crop image).

2. Is our choice of output layer activation and loss functions good considering the mentioned application? please discuss **why** and mention if something different is **better suited**? (2 pts)
- **Ans:** 

3.  Together, forward & backward pass, they form one training iteration known as epoch. Assume that we have a dataset with 400 rows/ samples and we randomly selected a batch size of 5 and 2000 epochs.
* 3a. How many batches there will be? (0.5 pts)
- **Ans:**
- 400/5 = 80
* 3b. How many batches will be considered to complete an epoch? (0.5 pts)
- **Ans:**
- All batches should pass through consider epoch complete. So 80 batches.
* 3c. After 2000 epochs, how many batches the model will pass through during the training process? (0.5 pts)
- **Ans:**
- 2_000*80 = 160_000

<font color='red'>**Explain all your calculations in details, if your answer is slightly wrongm but you reported correct logic, you still can get points**</font>

4. Imagine that you have a neural network with one hidden layer with 200 nodes (a "two-layer network", because it has two weight matrices), and you are using arctan activation function at the nodes. The inputs to the network are RGB images of size 64x64x3. The output is 5 probability values (obtained by passing the results through Softmax activation function). How many scalar operations (additions, subtractions, multiplications, divisions, exponentiations, arctan, logarithms and comparisons) does it take to perform a forward pass on one data point (to calculate the cross-entropy loss) (2.5 pts) _Hint! Beware of biases_
- **Ans:**


n * m multiplications
(n-1) * m summations 
complexity is O(max(n, m))

### Input parameters

D = 64*64*3
N = 200
C = 5

### First layer
XW1+b1

D*N + (D-1)*N + N

### Activation
N - each element we apply activation function arctan

### Second layer
HW2+b2

N*C + (N-1)*C + C

### Softmax
C (exponents) + C-1 (summations) + C (divisions)

### Total

sum all operations on each layer


5. What is the difference between stochastic gradient descent and batch gradient descent? (1 pts)
- **Ans:**
- Batch Gradient Descent can be used for smoother curves. SGD can be used when the dataset is large. Batch Gradient Descent converges directly to minima. SGD converges faster for larger datasets. (https://towardsdatascience.com/batch-mini-batch-stochastic-gradient-descent-7a62ecba642a)

6. Mention 1 application where calculating the gradient of the input is useful? (1 pts)
- **Ans:**
-